Test 3: kerchunk with MUR 0.01 degree data, creating kerchunk references and accessing data all in the cloud

To install kerchunk, used
```
!pip install git+https://github.com/fsspec/kerchunk

```

In [16]:
import fsspec
import kerchunk
from kerchunk.hdf import SingleHdf5ToZarr
from kerchunk.combine import MultiZarrToZarr
import ujson
import xarray as xr
import earthaccess

In [2]:
earthaccess.login()
granule_info = earthaccess.search_data(
    short_name="MUR-JPL-L4-GLOB-v4.1",
    temporal=("2019-01-01", "2019-12-31"),
    )

Enter your Earthdata Login username:  deanh808
Enter your Earthdata password:  ········


Granules found: 365


## For a single MUR file

In [3]:
fobj = earthaccess.open([granule_info[0]])[0]

Opening 1 granules, approx size: 0.37 GB
using endpoint: https://archive.podaac.earthdata.nasa.gov/s3credentials


QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
endpoint = fobj.full_name

In [5]:
%%time
reference = SingleHdf5ToZarr(fobj, endpoint, inline_threshold=0).translate()

CPU times: user 519 ms, sys: 172 ms, total: 690 ms
Wall time: 4.03 s


In [6]:
key = fobj.f.s3.key
secret = fobj.f.s3.secret
token = fobj.f.s3.token

In [7]:
%%time
data_ker = xr.open_dataset(
    "reference://", engine="zarr", chunks={},
    backend_kwargs={
        "storage_options": 
        {"fo": reference,
         "remote_protocol": "s3",
         "remote_options": {"anon": False, "key": key, "secret": secret, "token": token}},
         "consolidated": False
    }
)

CPU times: user 612 ms, sys: 12.2 ms, total: 624 ms
Wall time: 825 ms


In [8]:
data_ker

<xarray.Dataset> Size: 23GB
Dimensions:           (time: 1, lat: 17999, lon: 36000)
Coordinates:
  * lat               (lat) float32 72kB -89.99 -89.98 -89.97 ... 89.98 89.99
  * lon               (lon) float32 144kB -180.0 -180.0 -180.0 ... 180.0 180.0
  * time              (time) datetime64[ns] 8B 2019-01-01T09:00:00
Data variables:
    analysed_sst      (time, lat, lon) float64 5GB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    analysis_error    (time, lat, lon) float64 5GB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    dt_1km_data       (time, lat, lon) timedelta64[ns] 5GB dask.array<chunksize=(1, 1447, 2895), meta=np.ndarray>
    mask              (time, lat, lon) float32 3GB dask.array<chunksize=(1, 1447, 2895), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float64 5GB dask.array<chunksize=(1, 1447, 2895), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                CF-1.5
    Metadata_Conventions:       Unidata Observation Dataset v1.0
    acknowledgment:             Please acknowledge the use of these data with...
    cdm_data_type:              grid
    comment:                    MUR = "Multi-scale Ultra-high Reolution"
    creator_email:              ghrsst@podaac.jpl.nasa.gov
    ...                         ...
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    time_coverage_end:          20190101T210000Z
    time_coverage_start:        20181231T210000Z
    title:                      Daily MUR SST, Final product
    uuid:                       27665bc0-d5fc-11e1-9b23-0800200c9a66
    westernmost_longitude:      -180.0

In [9]:
%%time
data_ker['analysed_sst'].mean().compute()

CPU times: user 13.6 s, sys: 2.34 s, total: 16 s
Wall time: 9.4 s


<xarray.DataArray 'analysed_sst' ()> Size: 8B
array(286.95274384)

#### Open and process using netCDF backend for comparison

In [10]:
%%time
data_nc = xr.open_dataset(fobj)

CPU times: user 190 ms, sys: 3.31 ms, total: 193 ms
Wall time: 325 ms


In [11]:
%%time
data_nc['analysed_sst'].mean().compute()

CPU times: user 10.3 s, sys: 4.67 s, total: 14.9 s
Wall time: 18.5 s


<xarray.DataArray 'analysed_sst' ()> Size: 8B
array(286.95274384)

## Week of MUR files

#### Create single reference files for each day (7 total)

In [13]:
fobjs_week = earthaccess.open(granule_info[:7])

Opening 7 granules, approx size: 2.6 GB
using endpoint: https://archive.podaac.earthdata.nasa.gov/s3credentials


QUEUEING TASKS | :   0%|          | 0/7 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/7 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/7 [00:00<?, ?it/s]

In [14]:
endpoints_week = [fo.full_name for fo in fobjs_week]

In [15]:
%%time
refs_week = []
for fo in fobjs_week:
    refs_week.append(SingleHdf5ToZarr(fobj, fo.full_name, inline_threshold=0).translate())

CPU times: user 3.49 s, sys: 796 ms, total: 4.28 s
Wall time: 14 s


#### Then combine

In [19]:
key = fobjs_week[0].f.s3.key
secret = fobjs_week[0].f.s3.secret
token = fobjs_week[0].f.s3.token

In [20]:
%%time

mzz = MultiZarrToZarr(
    refs_week,
    remote_protocol="s3",
    remote_options={"anon": False, "key": key, "secret": secret, "token": token},
    concat_dims=["time"]
)

ref_week_combined = mzz.translate()

CPU times: user 95.5 ms, sys: 8.8 ms, total: 104 ms
Wall time: 1.22 s


#### And open the combined data set

In [21]:
data = xr.open_dataset(
    "reference://", engine="zarr", chunks={},
    backend_kwargs={
        "storage_options": {
            "fo": ref_week_combined,
            "remote_protocol": "s3",
            "remote_options": {"anon": False, "key": key, "secret": secret, "token": token}
        },
        "consolidated": False
    }
)
data

<xarray.Dataset> Size: 163GB
Dimensions:           (time: 7, lat: 17999, lon: 36000)
Coordinates:
  * lat               (lat) float32 72kB -89.99 -89.98 -89.97 ... 89.98 89.99
  * lon               (lon) float32 144kB -180.0 -180.0 -180.0 ... 180.0 180.0
  * time              (time) datetime64[ns] 56B 2019-01-01T09:00:00 ... 2019-...
Data variables:
    analysed_sst      (time, lat, lon) float64 36GB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    analysis_error    (time, lat, lon) float64 36GB dask.array<chunksize=(1, 1023, 2047), meta=np.ndarray>
    dt_1km_data       (time, lat, lon) timedelta64[ns] 36GB dask.array<chunksize=(1, 1447, 2895), meta=np.ndarray>
    mask              (time, lat, lon) float32 18GB dask.array<chunksize=(1, 1447, 2895), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float64 36GB dask.array<chunksize=(1, 1447, 2895), meta=np.ndarray>
Attributes: (12/47)
    Conventions:                CF-1.5
    Metadata_Conventions:       Unidata Observation Dataset v1.0
    acknowledgment:             Please acknowledge the use of these data with...
    cdm_data_type:              grid
    comment:                    MUR = "Multi-scale Ultra-high Reolution"
    creator_email:              ghrsst@podaac.jpl.nasa.gov
    ...                         ...
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    time_coverage_end:          20190101T210000Z
    time_coverage_start:        20181231T210000Z
    title:                      Daily MUR SST, Final product
    uuid:                       27665bc0-d5fc-11e1-9b23-0800200c9a66
    westernmost_longitude:      -180.0